# Extracting Information from Text Data Assignment

In [1]:
!pip install textacy --quiet

     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 143kB 48.4MB/s 
     |████████████████████████████████| 3.0MB 50.9MB/s 
     |████████████████████████████████| 481kB 45.8MB/s 


In [2]:
!pip install rake_nltk --quiet

In [0]:
import requests
from bs4 import BeautifulSoup
import spacy
import string
import textacy
import itertools
from nltk import pos_tag
from rake_nltk import Rake
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import tree2conlltags
from gensim.summarization import keywords
from nltk.chunk.regexp import RegexpParser
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

### Read the CNN Lite plain text file articles into a corpus using the NLTK's PlaintextCorpusReader.

In [0]:
articles = ['http://lite.cnn.io/en/article/h_eac18760a7a7f9a1bf33616f1c4a336d',
            'http://lite.cnn.io/en/article/h_de3f82f17d289680dd2b47c6413ebe7c',
            'http://lite.cnn.io/en/article/h_72f4dc9d6f35458a89af014b62e625ad',
            'http://lite.cnn.io/en/article/h_aa21fe6bf176071cb49e09d422c3adf0',
            'http://lite.cnn.io/en/article/h_8ad34a532921c9076cdc9d7390d2f1bc',
            'http://lite.cnn.io/en/article/h_84422c79110d9989177cfaf1c5f45fe7',
            'http://lite.cnn.io/en/article/h_d010d9580abac3a44c6181ec6fb63d58',
            'http://lite.cnn.io/en/article/h_fb11f4e9d7c5323e75b337d9e9e5e368',
            'http://lite.cnn.io/en/article/h_7b27f0b131067f8ece6238ac559670ab',
            'http://lite.cnn.io/en/article/h_8cae7f735fa9573d470f802063ceffe2',
            'http://lite.cnn.io/en/article/h_72c3668280e82576fcc2602b0fa70c14',
            'http://lite.cnn.io/en/article/h_d20658fb0e20212051cda0e0a7248c8a',
            'http://lite.cnn.io/en/article/h_56611c43d7928120d2ae21666ccc7417',
            'http://lite.cnn.io/en/article/h_bda0394e3c5ee7054ee65c022bca7695']

In [0]:
TAGS = ['p']
data = []

for article in articles:
    response = requests.get(article)
    content = response.text
    soup = BeautifulSoup(content)
    text_list = [tag.get_text() for tag in soup.find_all(TAGS)]
    text = ' '.join(text_list)
    data.append(text)

In [40]:
!mkdir news_articles

mkdir: cannot create directory ‘news_articles’: File exists


In [0]:
for i, d in enumerate(data):
    file = open(f'./news_articles/article_{i}.txt', 'w')
    file.write(data[i])
    file.close()

In [8]:
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader
import nltk
nltk.download('punkt')
DOC_PATTERN = r'.*\.txt'
CAT_PATTERN = r'([\w_\s]+/.*)'

corpus = PlaintextCorpusReader('news_articles', DOC_PATTERN)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Iterate through the fileids in the corpus, extract the raw text of each document, and store them in a list.

In [9]:
doc_list = []
for fileid in corpus.fileids():
    doc = corpus.raw(fileid)
    if len(doc)>0:
        doc_list.append(doc)

doc_list

['(CNN) - Pink has been working pretty hard and it sounds like she will be taking a step back in 2020. Speaking with "Entertainment Tonight" on the Country Music Association Awards red carpet, the singer was joined by her husband, Carey Hart, and their kids Willow, 8, and Jameson, 2. Pink was there to perform her song "Love Me Anyway" with country star Chris Stapleton, and she  talked about how hectic things have been.  "We did two and a half years of [music] and Willow\'s back in school now, Jameson\'s going to start pre-school soon," Pink said. "It\'s kind of the year of the family." The star also praised her husband, with whom she will celebrate 14 years of marriage in January. "Carey has a lot going on as well," she said of Hart, who went from being a professional motocross competitor to racing off-road trucks. "He\'s super supportive, he follows me around the world and now it\'s his turn." According to Billboard, Pink\'s Beautiful Trauma Tour ranks as the 10th highest-grossing tou

### Extract the top 5 keywords from every document in the corpus. Print them and compare the differences in keywords among the documents.

In [0]:
keywords5 = []
for doc in doc_list:
    key_words = keywords(doc,words=5, lemmatize=True).split('\n')
    keywords5.append(key_words)

In [11]:
keywords5

[['pink', 'carey', 'tour', 'cnn', 'pretty'],
 ['patrick', 'primary', 'news', 'cnn', 'democrats'],
 ['tail', 'narwhal', 'puppy', 'unicorn', 'dogs'],
 ['states', 'democrats', 'news', 'bloomberg', 'told'],
 ['republican', 'taylor', 'presidents', 'rep', 'ukraine'],
 ['muslimness', 'people', 'skin', 'white', 'religion'],
 ['news', 'said', 'trump', 'republican', 'media'],
 ['said', 'brown', 'jones', 'police', 'roanoke'],
 ['trump', 'hotels', 'office', 'profit', 'owned'],
 ['keys', 'grammys', 'award', 'wanna', 'reserved'],
 ['americans', 'republican', 'ukrainians', 'investigate', 'impeachable'],
 ['student', 'said', 'told', 'pence', 'schools'],
 ['crows', 'cnn', 'american', 'old', 'content'],
 ['protester', 'police', 'news', 'chinese', 'including']]

In [45]:
max(keywords5), min(keywords5)

(['trump', 'hotels', 'office', 'profit', 'owned'],
 ['americans', 'republican', 'ukrainians', 'investigate', 'impeachable'])

### Extract the top 3 keyphrases from each document, print them, and compare the differences.

In [0]:
keyphrases3 = []
for doc in doc_list:
    r = Rake()
    r.extract_keywords_from_text(doc)
    key_phrases = r.get_ranked_phrases_with_scores()
    keyphrases3.append(key_phrases)

In [56]:
key_phrases[:3]

[(31.5, 'fall semester around two weeks early'),
 (31.066666666666663, 'senior chinese official shen chunyao said'),
 (29.92857142857143, 'three tunnels connecting hong kong island')]

### Identify and extract the named entities in each document, filtering out the numeric types. Print them and compare the differences between documents.

In [0]:
nlp = spacy.load('en_core_web_sm')
spacy_doc = nlp(doc)
entities = [[entity.text, entity.label_] for entity in spacy_doc.ents]

In [15]:
print(entities)

[['Hong Kong', 'GPE'], ['CNN', 'ORG'], ['Hong Kong', 'GPE'], ['Thursday', 'DATE'], ['evening', 'TIME'], ['thousands', 'CARDINAL'], ['six-month-long', 'DATE'], ['recent days', 'DATE'], ['Chinese', 'NORP'], ['Thursday morning', 'TIME'], ['the Hong Kong Polytechnic University', 'ORG'], ['Kowloon', 'GPE'], ['Hong Kong Island', 'GPE'], ['the University of Hong Kong', 'ORG'], ['the Chinese University of Hong Kong', 'ORG'], ['the New Territories', 'GPE'], ['several thousand', 'CARDINAL'], ['the third straight day', 'DATE'], ['Tuesday', 'DATE'], ['June', 'DATE'], ['hundreds', 'CARDINAL'], ['more than 1,567', 'CARDINAL'], ['Wednesday', 'DATE'], ['Thursday', 'DATE'], ['Hong Kong', 'GPE'], ['around two weeks early', 'DATE'], ['Wednesday', 'DATE'], ['China', 'GPE'], ['Thursday', 'DATE'], ['Central district', 'LOC'], ['a fourth straight day', 'DATE'], ['Tai Koo', 'GPE'], ['Two', 'CARDINAL'], ['three', 'CARDINAL'], ['Hong Kong Island', 'GPE'], ['Cross-Harbour Tunnel', 'ORG'], ['Thursday', 'DATE'], [

In [57]:
max(entities), min(entities)

(['three', 'CARDINAL'], ['20', 'CARDINAL'])

In [58]:
from spacy import displacy
displacy.serve(spacy_doc,style='ent')


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


### Use NLTK's RegexpParser to extract all instances of two or more consecutive nouns (of any tense).

In [0]:
grammar = r'KT: {(<NN>|<NNS>|<NNP>|<NNPS>)(<NN>|<NNS>|<NNP>|<NNPS>)+}'

def extract_phrases(doc,grammar):
    chunker = RegexpParser(grammar)
    results = []

    for sent in sent_tokenize(doc):
        tokenized = word_tokenize(sent)
        tagged = pos_tag(tokenized)
        cleaned = [tag for tag in tagged if not tag[1] in string.punctuation]
        chunks = tree2conlltags(chunker.parse(cleaned))
        phrases = [' '.join(word for word, pos, chunk in group) 
            for key, group in groupby(chunks,lambda term: term[-1] !='o') if key]

        results += phrases
        results = list(set(results))
    return results

In [68]:
import nltk
from itertools import groupby
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [78]:

for doc in doc_list:
    x = extract_phrases(doc,grammar)
    for item in x:
        print(item)

`` We did two and a half years of [ music ] and Willow 's back in school now Jameson 's going to start pre-school soon '' Pink said
`` He 's super supportive he follows me around the world and now it 's his turn ''
CNN Pink has been working pretty hard and it sounds like she will be taking a step back in 2020
The star also praised her husband with whom she will celebrate 14 years of marriage in January
Pink was there to perform her song `` Love Me Anyway '' with country star Chris Stapleton and she talked about how hectic things have been
© 2019 Cable News Network
`` It 's kind of the year of the family ''
Turner Broadcasting System Inc. All Rights Reserved
Speaking with `` Entertainment Tonight '' on the Country Music Association Awards red carpet the singer was joined by her husband Carey Hart and their kids Willow 8 and Jameson 2
`` Carey has a lot going on as well '' she said of Hart who went from being a professional motocross competitor to racing off-road trucks
Listen to CNN low

### For every document in the corpus, iterate over every sentence, extract any SVO triples, print them, and compare.

In [0]:
results = []
for sent in spacy_doc.sents:
    svo = textacy.extract.subject_verb_object_triples(sent)
    results += svo
results = list(set(results))

In [36]:
print(results)

[(We, have had, warnings), (who, are coerced, to be), (what, is believed, to be), (it, is, question), (protesters, barricaded, themselves), (Looming, is, specter), (they, had evacuated, students), (state media, warning, protesters), (level, has reached, heights), (protesters, to cross, it), (We, have had, talking), (stop, has, to be implemented), (Beijing, has, line), (students, have been asked, to leave), (protesters, seek, to breach), (government supporters, hurling, bricks), (television channel, issued, editorial), (We, have had, persuasion), (country, will never accept, situation), (you, are going, to have), (police, fired, gas), (protesters, hurling, bricks), (you, can make, call), (thousands, prepared, to face), (protesters, continued, to pour), (government, unveiled, range), (stop, has, advanced), (It, echoed, editorial), (MTR, continued, to suspend), (protests, have taken, turn), (escalation, raised, questions), (there, is, likelihood), (We, warn, protesters), (It, 's, not to s

In [76]:
max(results), min(results)

((Buried, is, call), (thousands, prepared, to face))